# **Customer Retention Toolkit Demo**

## Introduction
Welcome to our Customer Churn Prediction guide. Within the competitive subscription service industry, addressing customer churn is key to business longevity. Utilizing marketing analytics and machine learning, this project predicts churn from customer data and behavior. This guide outlines use cases like early risk detection and tailored retention campaigns, aiming to lower churn and boost customer contentment for enduring business growth.

## Package Structure Overview

- **api**: Contains the interface for interacting with the predictive model.
- **db**: Database scripts, including the schema and SQL interactions.
- **docs**: Documentation files, such as the Entity Relationship Diagram (ERD), usage instructions, roadmap, and use cases.
- **logger**: Logging functionality to keep track of operations and errors.
- **models**: The machine learning models and related workflows.


## Setup and Configuration
Steps to run the code:

1. Create new env. (pythom -m venv venv)
2. Install the requirements.txt in the env by.(pip install -r requirements.txt)
3. Run the cells in example.ipynb
4. When you reach the API part, start the api (python run.py)

The following steps are for web usage of API:

5. Then next of http://127.0.0.1:5000 add http://127.0.0.1:5000/docs and press enter.
6. Press on the get_info, theb press on try it out and on the ID write any id from 1 to 3 and press execute.
7. you will find the result in Response body. 

## Importing the libraries

``` python

import pandas as pd
import os
import random
from customer_retention_toolkit.db.schema import *
from customer_retention_toolkit.db.sql_interactions import  SqlHandler
from customer_retention_toolkit.logger import *
from FillTables import *
from customer_retention_toolkit.models.MLWorkflow import MLWorkflow
from customer_retention_toolkit.api import app

# Configure the logger
logger = logging.getLogger('example_notebook')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(ch)
```

## **Data Preparation and Ingestion** (Database)

db includes the following:

- **schema.py**:  defines the structure of tables and creates them 
- **sql_interactions**: for all interactions with tables, querries 

``` python
db_filename = 'temp.db'

if os.path.exists(db_filename):
    os.remove(db_filename)
    logger.info(f"Deleted existing {db_filename}")

create_database()
InsertToTables()
```

## Model
Includes: 
- **MLWorkflow.py**: Whole pipeline of ML, including from getting data from db, preprocessing, prediction, adding prediction to db

### **Load data from the database**

``` python
dbname = 'temp'
workflow = MLWorkflow(dbname)
```

### **Run the workflow**

``` python
table_names = ['State', 'PlanDetails', 'DayUsage', 'EveUsage', 'NightUsage', 'IntlUsage', 'CustomerMetrics']
metrics, X_test, y_test, best_model_predictions = workflow.run_workflow(table_names)

logger.info("Random Forest Model Metrics:")
for metric_name, metric_value in metrics.items():
    logger.info(f"{metric_name}: {metric_value}")
```

### **Save predictions to the database**

``` python
workflow.save_predictions_to_db(X_test, y_test, best_model_predictions, table_name='PredictionResults')    
```

## **Example API Test**

### API Usage

- **api.py**: api functionality such as get_data, create_data, update_data, predict_churn
- **run.py**: for starting the api


``` python
import requests

# The base URL for your API
base_url = "http://127.0.0.1:5000"

# GET request to the root endpoint
response = requests.get(f"{base_url}/")
print(response.json())
```

``` python
# Replace with a valid customer ID
customer_id = 1
response = requests.get(f"{base_url}/get_data/{customer_id}")
print(response.json())
```

``` python
# base_url = 'http://127.0.0.1:5000'  # Replace with the actual base URL if different

new_customer_data = {
    "CustomerID": 2749,
    "ChurnStatus": 1,
    "StateID": 1,  # Assuming '1' is a valid StateID in your database
    "PlanID": 1,  # Assuming '1' is a valid PlanID in your database
    "DayUsageID": 1,  # Assuming '1' is a valid DayUsageID in your database
    "EveUsageID": 1,  # Assuming '1' is a valid EveUsageID in your database
    "NightUsageID": 1,  # Assuming '1' is a valid NightUsageID in your database
    "IntlUsageID": 1,  # Assuming '1' is a valid IntlUsageID in your database
    "CustomerServiceCalls": 1  # Number of customer service calls
}

response = requests.post(f"{base_url}/create_data", json=new_customer_data)
print(response.json())
```

``` python
# PUT request to update a customer record
update_data = {
    "column_name": "ChurnStatus",
    "new_value": 1,
    "CustomerID": 2749  # Make sure this ID exists in your database
}
response = requests.put(f"{base_url}/update_data", json=update_data)
print(response.json())
```

``` python
# GET request to predict churn for a customer
# response = requests.get(f"{base_url}/predict_churn/{customer_id}")
# print(response.json())
customer_id = 555
response = requests.get(f"{base_url}/predict_churn/{customer_id}")
print('Status Code:', response.status_code)

if response.status_code == 200:
    try:
        data = response.json()
        print(data)
    except JSONDecodeError:
        print('Response could not be decoded as JSON:', response.text)
else:
    print('Failed to fetch data:', response.text)
```